In [19]:
import sys
import pandas as pd
import numpy as np
import plotly.express as px
from sqlalchemy import create_engine

sys.path.append('../')

DB_URI = "postgresql+psycopg2://xodh3@localhost:5432/economy_db"
engine = create_engine(DB_URI)

print("🚀 준비 완료!")

🚀 준비 완료!


In [20]:
#날짜순으로 정렬해서 가져오기
query = """
SELECT trade_date, ticker, close_price
FROM practice_spy
ORDER BY trade_date ASC
"""
df = pd.read_sql(query, engine)

# 피벗: 보기 좋게 가로로 펼치기
df_pivot = df.pivot(index='trade_date', columns='ticker', values='close_price')
df_pivot = df_pivot.bfill().dropna()

print("데이터 로딩 완료:")
print(df_pivot.tail())


데이터 로딩 완료:
ticker         GLD     QQQ     SPY      TLT
trade_date                                 
2025-11-24  380.08  608.89  675.02  89.9152
2025-11-25  383.12  614.27  679.68  90.3138
2025-11-27  387.88  619.25  683.39  89.8853
2025-11-30  389.75  617.17  680.27  88.7700
2025-12-01  387.24  622.00  681.53  88.8100


In [21]:
# 일반 수익율
returns = df_pivot.pct_change().dropna()
# 🧮 NumPy 연산: 연간 변동성(위험도) 계산  np.std(returns): 일일 표준편차 (Daily Volatility)
# 금융에서 변동성은 투자 기간을 비교하기 위해 연간 단위로 표시하는 것이 일반적인 관례입니다.
# 252는 1년 동안의 평균 거래일 수를 나타내는 일반적인 값입니다 (공휴일과 주말을 제외).
# 변동성은 시간의 제곱근에 비례하여 증가한다는 통계적 원칙인 **'시간의 제곱근 법칙(Square Root of Time Rule)'**에 따라, 일일 표준편차에 np.sqrt(252) (252의 제    곱근)를 곱하여 연간 변동성으로 환산합니다.
# * 100: 백분율 변환
# 계산된 변동성 값은 소수점 형태(예: 0.20)이므로, 이를 더 직관적인 **백분율(%)**로 표시하기 위해 100을 곱합니다.
# 결과적으로 0.20은 "연간 변동성 20%"를 의미하게 됩니다.

volatility = np.std(returns, axis=0) * np.sqrt(252) * 100

print("=== 📊 자산별 위험도 (숫자가 클수록 롤러코스터) ===")
print(volatility.sort_values(ascending=False))


=== 📊 자산별 위험도 (숫자가 클수록 롤러코스터) ===
ticker
QQQ    23.918474
SPY    19.726883
GLD    19.427925
TLT    12.232103
dtype: float64


In [22]:
# 100을 기준으로 지수화 (시작점을 똑같이 맞춰서 비교)
normalized_df = (df_pivot / df_pivot.iloc[0]) * 100

fig = px.line(normalized_df, x= normalized_df.index, y=normalized_df.columns,
              title='<b>자산 배분 4대장 수익률 비교 (Start=100)</b>',
              labels={'value': '성장률 (100 -> ?)', 'trade_date': '날짜', 'variable': '자산'})

# 마우스 오버 시 싹 다 보여주기
fig.update_layout(hovermode="x unified")
fig.show()